In [605]:

c_ = 0.1 * tf.Variable(tf.truncated_normal([4,4], dtype=tf.float32))
h_ = 0.1 * tf.Variable(tf.truncated_normal([4,4], dtype=tf.float32))

In [10]:
import numpy as np
import tensorflow as tf
sess = tf.Session()

x = [[1,0,0,0],# 4*4  #h
        [0,1,0,0], #e
        [0,0,1,0], #l
        [0,0,0,1]] #o

y = [[0,1,0,0], # 5*4
        [0,0,1,0],
        [0,0,1,0],
        [0,0,0,1]]
c_ = 0.1 * tf.constant([[1,0,0,0], # 4*4
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]], dtype=tf.float32)

h_ = 0.1 * tf.constant([[1,0,0,0], # 4*4
                       [0,1,0,0],
                       [0,0,1,0],
                       [0,0,0,1]], dtype=tf.float32)
c_ = 0.1 * tf.Variable(tf.truncated_normal([4,4], dtype=tf.float32))
h_ = 0.1 * tf.Variable(tf.truncated_normal([4,4], dtype=tf.float32))

#he
#el
#ll
#lo

seq_len = len(x)#cell 갯수, 인풋이 몇 덩어리인지
num_units = 4# hiddenlayer 갯수, c값이 몇 덩어리인지

#seq_len = len(sess.run(x))#cell 갯수, 인풋이 몇 덩어리인지
#num_units = len(sess.run(c_))# hiddenlayer 갯수, c값이 몇 덩어리인지
#x행 = h행
#c행 = 노상관
#h행 = c행
##########달라도 되는건 x열뿐############
class lstm:
    def build(c, h):
        args = tf.concat((x,h), axis=1)
#        print(args)

        out_size = 4 * num_units
        proj_size = args.shape[-1]
#        print(out_size)
#        print(proj_size)

        weights = tf.ones([proj_size, out_size]) * 0.5
#        print(weights)


        out = tf.matmul(args, weights)
#        print(out)

        bias = tf.ones([out_size]) * 0.5
#        print(bias)

        concat = out + bias
#        print(concat)

        i, j, f, o = tf.split(concat, 4, 1)
#        print(i)
#        print(j)
#        print(f)
#        print(o)

        g = tf.tanh(j)
#        print(g)

        def sigmoid_array(x):
            return 1 / (1 + tf.exp(-x))

        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)
#        print(sigmoid_f)

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g
#        print(new_c)

        new_h = tf.tanh(new_c) * sigmoid_array(o)
#        print(new_h)

#        print('\n new_h:',new_h)
#        print('\n new_c',new_c)

#        print(res[1].h)
#        print(res[1].c)

        return new_c, new_h

In [11]:
#ta = tf.TensorArray(size=1, dtype=tf.float32)
ta_c = tf.TensorArray(size=seq_len, dtype=tf.float32)
ta_h = tf.TensorArray(size=seq_len, dtype=tf.float32)

In [12]:
def body(last_state, last_output, step):
    
    output = lstm.build(last_state, last_output)[0]
    state = lstm.build(last_state, last_output)[1]


    return state, output, tf.add(step, 1)
    

timesteps = seq_len

steps = lambda a, b, step: tf.less(step, timesteps)

lstm_output, lstm_state, step = tf.while_loop(steps, body, (c_, h_, 0), parallel_iterations=20)


def body(last_state, last_output, step, ta_c, ta_h):
    
    output = lstm.build(last_state, last_output)[0]
    state = lstm.build(last_state, last_output)[1]
    ta_c = ta_c.write(step, state)
    ta_h = ta_h.write(step, output)
    return state, output, tf.add(step, 1), ta_c, ta_h
    

timesteps = 100

steps = lambda a, b, step, c, d: tf.less(step, timesteps)

lstm_output, lstm_state, step, ta_c, ta_h = tf.while_loop(steps, body, (c_, h_, 0, ta_c, ta_h), parallel_iterations=20)


In [13]:
xdata = [[1,0,0,0],# 4*4  #h
        [0,1,0,0], #e
        [0,0,1,0], #l
        [0,0,0,1]] #o

ydata = [[0,1,0,0], # 5*4
        [0,0,1,0],
        [0,0,1,0],
        [0,0,0,1]]

X = tf.placeholder(dtype=tf.float32, shape=[None, 4])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 4])

W = tf.Variable(tf.random_normal([4, 1]))
b = tf.Variable(tf.random_normal([1]))

logits = tf.matmul(sess.run(sestf.while_loop(steps, body, (c_, h_, 0), parallel_iterations=20)[0]), W) + b

FailedPreconditionError: Attempting to use uninitialized value Variable_15
	 [[Node: Variable_15/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_15)]]

Caused by op 'Variable_15/read', defined at:
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\ipykernel\kernelapp.py", line 497, in start
    self.io_loop.start()
  File "C:\Users\tjdwo\Anaconda3\envs\tf1.8\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asy

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train = tf.train.AdamOptimizer(1).minimize(cost)

In [643]:
is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [644]:
sess.run(tf.global_variables_initializer())

for i in range(3):
    sess.run(tf.while_loop(steps, body, (c_, h_, 0, ta_c, ta_h), parallel_iterations=20), feed_dict={X:xdata, Y:ydata})
    a,c  = sess.run([accuracy,cost],feed_dict={X:xdata, Y:ydata})
    print("##########################",a)
    print("##########################",c)

In [645]:
for i in range(3):
    sess.run(tf.while_loop(steps, body, (c_, h_, 0), parallel_iterations=20), feed_dict={X:xdata, Y:ydata})
    a,c  = sess.run([accuracy,cost],feed_dict={X:xdata, Y:ydata})
    print("##########################",a)
    print("##########################",c)

########################## 0.0
########################## 1.3862944
########################## 0.0
########################## 1.3862944
########################## 0.0
########################## 1.3862944


In [ ]:
sess.run(lstm_output[1])

In [ ]:
sess.run(logits)